# Image registration for Altum Macroalgae 
As it is a Altum orthomosaic, scaled from 0-to-1 and the filetitles gives 'reflectance', we assume it is calibrated
* For the following Dataset: https://zenodo.org/records/13880699
* Using code snippets from:
  
Output: `uint16` 0-65535 scaled .tif files with green, red, rededge, nir bands


In [1]:
import os
import rioxarray as rxr
import tifffile
from glob import glob
import numpy as np
from itertools import repeat

In [3]:
def load_image(path):
    return rxr.open_rasterio(path), path
    
def rescale_to_uint16(da):
    da = da.where(~np.isnan(da), 65535)
    rescaled = (da * 65535).clip(0, 65535).astype(np.uint16)
    
    return rescaled

    
def create_4_channel_image(green, red,rededge, nir):
    """Create a 4-channel image (G, R, RE, NIR)."""
    return np.stack((green, red, rededge, nir), axis=-1)
    

def save_image(filename, image, metadata=None):
    """Save the image as a 4-channel TIFF using tifffile."""
    #tifffile.imwrite(filename, image, photometric='rgb',extratags=metadata[0],subifds = metadata[1])
    if metadata is not None:
        extratags = metadata[0]
        subifds = metadata[1]
        with tifffile.TiffWriter(filename) as tiff:
            tiff.write(
                image,
                photometric='rgb',
                extratags=extratags,
                subifds=subifds
            )
    else:
        with tifffile.TiffWriter(filename) as tiff:
            tiff.write(
                image,
                photometric='rgb')
            


def process_multispec_set(green_image_path, output_directory):

    base_name = os.path.basename(green_image_path).replace("_green.tif", "")
    # Load the green band image (reference image)
    green_image, green_metadata = load_image(green_image_path)
    
    bands = {}
    for band in ["blue", "red", "red edge", "nir"]:
        band_image_path = green_image_path.replace("_green.tif", f"_{band}.tif")
        if os.path.exists(band_image_path):
            bands[band], _ = load_image(band_image_path)
        else:
            print(f"Missing {band} band for {green_image_path}")
            return
    bands["green"] = green_image
        
    sensor = "ALTUM"
    cal = "CAL"
    set_title = os.path.basename(os.path.normpath(output_directory))
    output_filename = f"{sensor}_{cal}_{set_title}_{base_name}.tif"
    output_path = os.path.join(output_directory, output_filename)
    os.makedirs(output_directory, exist_ok=True)

    # processing steps:
    fill = bands["green"]._FillValue
    blue = rescale_to_uint16(bands["blue"]) 
    green = rescale_to_uint16(bands["green"]) 
    red = rescale_to_uint16(bands["red"])  
    rededge = rescale_to_uint16(bands["red edge"]) 
    nir = rescale_to_uint16(bands["nir"]) 
    

     # Create a 4-channel image
    image_4ch = np.stack((blue,green, red, rededge, nir), axis=-1)
    if len(image_4ch.shape)>3:
        image_4ch = np.squeeze(image_4ch)
        print("Squeezed it")

    # Save the 4-channel image
    save_image(output_path, image_4ch)

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def process_directory(input_dir, output_directory, n_threads= 1):
    """Process an entire directory with multiple sets of multispectral images."""
    input_dir = os.path.abspath(input_dir)
    os.makedirs(output_directory, exist_ok=True)

    # Find all green band images (reference images)
    green_images = glob(os.path.join(input_dir, "*_green.tif"))

    # Process sets of 4 multispectral images in parallel with a progress bar
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        list(tqdm(executor.map(process_multispec_set, green_images, repeat(output_directory)), total=len(green_images)))

    print("All images successfully processed.")

In [4]:
folder = "../../data/MS_pretraining/Altum_macroalgae/"
orthos = os.path.join(folder, "Orthoimages")

In [5]:

input_folder = orthos
output_folder = "../../data/processed/altum_macroalgae"
process_directory(input_folder, output_folder)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

Squeezed it


 25%|█████████████████████                                                               | 1/4 [00:17<00:51, 17.17s/it]

Squeezed it


 50%|██████████████████████████████████████████                                          | 2/4 [00:31<00:30, 15.48s/it]

Squeezed it


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:46<00:15, 15.42s/it]

Squeezed it


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:02<00:00, 15.63s/it]

All images successfully processed.
